<a href="https://colab.research.google.com/github/jmhuer/utaustin_optimization/blob/main/homework6/homework6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 6
In this problem set you will implement SGD and SVRG and compare the two to each other, and also to GD.

In [1]:
!pip install gdown
! git clone https://github.com/jmhuer/optimization_tools
from optimization_tools.utils import download_gdrive

# we pass the ID between two slashes '/{ID}/'  
digits = '1X4fvOpQ2LK81-D3Fw4Oxugdvgm1dDJ3d'
news = '1Sc6ew-Ti8uNAPScD9P1UXPXrziPqaEwd'
download_gdrive(digits)
download_gdrive(news)


fatal: destination path 'optimization_tools' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=1X4fvOpQ2LK81-D3Fw4Oxugdvgm1dDJ3d
To: /content/digits.zip
100%|##########| 98.7k/98.7k [00:00<00:00, 38.2MB/s]

Downloading...
From: https://drive.google.com/uc?id=1Sc6ew-Ti8uNAPScD9P1UXPXrziPqaEwd
To: /content/news.zip
8.53MB [00:00, 51.8MB/s]



# Problem 1: Stochastic Variance Reduced Gradient Descent (SVRG)

As we discussed in the video lectures, decomposable functions of the form
$$
\min_{\omega} \left [ F(\omega) = \frac{1}{n} \sum_i^n f_i(\omega) \right ],
$$
are very common in statistics/ML problems. Here, each $f_i$ corresponds to a loss for a particular training example. For
example, if $f_i(\omega) = (\omega^\top x_i - y_i)^2$, then $F(\omega)$ is a least
squares regression problem. The standard gradient descent (GD) update 
$$
\omega_t = \omega_{t-1} - \eta_t \nabla F(\omega_{t-1})
$$

evaluates the full gradient $\nabla F(\omega) = \frac{1}{n} \sum_i^n \nabla
f_i(\omega)$, which requires evaluating $n$ derivatives. This can be
prohibitively expensive when the number of training examples $n$ is large. SGD evaluates
the gradient of one (or a small subset) of the training examples--drawn
randomly from ${1,...n}$--per iteration:
$$
\omega_t = \omega_{t-1} - \eta_t \nabla f_i(\omega_{t-1}).
$$

In expectation, the updates are equivalent, but SGD has the computational
advantage of only evaluating a single gradient $\nabla f_i(\omega)$. The
disadvatage is that the randomness introduces variance, which slows
convergence. This was our motivation in class to introduce the SVRG algorithm.

Given the dataset in **digits.zip**, plot the performance of GD, SGD, and SVRG for logistic regression with $l2$ regularization in terms of negative log likelihood on the training data against the number of gradient evaluations for a single training example (GD performs $n$ such evaluations per iteration and SGD performs $1$). Choose the $l2$ parameter to optimize performance on the test set. How does the choice of $T$ (the number of inner loops) affect the performance of SVRG? There should be one plot with a title and three lines with different colors, markers, and legend labels.



In [2]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import time
import pdb
from tqdm import tqdm
import zipfile as zipfile

%matplotlib inline

#sample code to load digits.zip
def loaddata(filename):
    data={}
    with zipfile.ZipFile(filename) as z:
        for filename in z.namelist():
          data[filename] = pd.read_csv(z.open(filename), sep=' ', header=None)
    return data

digits_dict = loaddata('./digits.zip')
print(digits_dict.keys())
X_digits_train = digits_dict['X_digits_train.csv']
X_digits_test = digits_dict['X_digits_test.csv']
y_digits_train = digits_dict['y_digits_train.csv'].to_numpy(dtype=int).ravel()
y_digits_test = digits_dict['y_digits_test.csv'].to_numpy(dtype=int).ravel()

dict_keys(['X_digits_test.csv', 'X_digits_train.csv', 'y_digits_test.csv', 'y_digits_train.csv'])


In [3]:
import numpy as np
import numpy.random as rn
import numpy.linalg as la
import matplotlib.pyplot as plt
import time
import plotly.graph_objects as graph
import numpy.linalg as la
from tqdm import tqdm

class MSE:
    def __init__(self, train_data, test_data):
        A ,b = train_data['data'], train_data['labels']
        self.loss = lambda weight : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weight)-b)**2)
        self.grad_loss = lambda weight : (1./b.shape[0])*np.dot(A.T,np.dot(A,weight)-b)
        #test loss
        A_test ,b_test = test_data['data'], test_data['labels']
        self.test_loss = lambda weights : (1./b_test.shape[0])*(.5)*np.sum((np.dot(A_test,weights)-b_test)**2)
    def eval(self, weight):
        return self.loss(weight) 
    def gradient(self,weight):
        return self.grad_loss(weight) 
    def test_eval(self, weights):
        return self.test_loss(weights) 


class SVRG:
    def __init__(self, A, b, lmda = 1e-3):
        self.lmda = lmda
        self.loss = lambda weight : (1./b.shape[0])*(.5)*np.sum((np.dot(A,weight)-b)**2)
        self.reg  = lambda weight : self.lmda*la.norm(weight,1)
        self.grad_loss = lambda weight : (1./b.shape[0])*np.dot(A.T,np.dot(A,weight)-b)
        self.grad_reg  = lambda weight : self.lmda*la.norm(weight,1) 
    def eval(self, weight):
        return self.loss(weight) 
    def gradient(self,weight):
        return self.grad_loss(weight) 

  

In [4]:
class GD:
    def __init__(self, function, x_init, epochs, learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.history = {"step": [0],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], 
                        "opt_dif": [],
                        "test_loss": []}
    def go(self, test = False):
        for i in range(self.epochs):
          ##RETRIVE OLD VALS
          x_old = self.history['x_vals'][-1]
          ##EVALUATE HERE
          y = self.function.eval(x_old)
          g = self.function.gradient(x_old)
          ##UPDATE HERE 
          x = x_old - self.learning_rate(i) * g
          ##STORE HISTORY
          self.history['step'].append(self.history['step'][-1] + 1)
          self.history['function_vals'].append(float(y))
          self.history['grad_vals'].append(g)
          self.history['x_vals'].append(x)
          if test:  
              self.history['test_loss'].append(float(self.function.test_eval(x)))
          ##CLEAN UP FOR SPARSE CASE
          del self.history['grad_vals'][0:1]
          del self.history['x_vals'][0:1] 

In [5]:
class SGD:
    def __init__(self, function, x_init, epochs, learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.history = {"step": [],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [self.x_init], 
                        "opt_dif": [],
                        "test_loss": []}      
    def go(self, test = False):
        for i in tqdm(range(self.epochs)):
          for j in range(self.batch_size):
              ##RETRIVE OLD VALS
              x_old = self.history['x_vals'][-1]
              ##EVALUATE HERE
              y = self.function.eval(x_old)
              g = self.function.gradient(x_old)
              ##UPDATE HERE 
              x = x_old - self.learning_rate(i) * g
              ##STORE HISTORY
              self.history['step'].append(i)
              self.history['function_vals'].append(float(y))
              self.history['grad_vals'].append(g)
              self.history['x_vals'].append(x)
              self.history['grad_vals'].append(g)
              self.history['x_vals'].append(x)
              if test:  
                  self.history['test_loss'].append(float(self.function.test_eval(x)))
              ##CLEAN UP FOR SPARSE CASE
              del self.history['grad_vals'][0:1]
              del self.history['x_vals'][0:1] 

In [59]:

class SGD2:
    class mydataset(Dataset): #accepts numpy
      def __init__(self, data, label, transform=None):
        self.data = data
        self.label = label
      def __len__(self):
        return len(self.label)
      def __getitem__(self, idx):
        data, label  = self.data[idx,:], self.label[idx]
        return data, label 
    def __init__(self, function, x_init, epochs, batch_size,learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.history = {"step": [0],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [np.zeros((64,10))], 
                        "opt_dif": [],
                        "test_loss": []}      
    def go(self, dataloader, test = False):
        for _ in tqdm(range(self.epochs)):
            for i, (data, label) in enumerate(dataloader):
                train_data = {"data"  : data.numpy(),
                              "labels": label.numpy()}

                gd  = GD(Logistic_loss(train_data = train_data, test_data = test_data), x_init= None, epochs = 1, learning_rate = lambda s : 0.001)
                gd.history = self.history.copy()
                gd.go(test = test)
                self.history = gd.history.copy()




In [60]:
import scipy
from scipy.sparse.linalg import norm
from torch.utils.data import Dataset, DataLoader



def L_M(A):
  sm , sc = (1/A.shape[0])*la.norm(A,2)**2,(1/A.shape[0])*la.norm(A,-2)**2
  return sm , sc

train_data = {"data"  : X_digits_train.to_numpy(),
              "labels": y_digits_train}

test_data  = {"data"  : X_digits_test.to_numpy(),
              "labels": y_digits_test}

sm , sc = L_M(X_digits_train)
gd    = GD(Logistic_loss(train_data, test_data), x_init= np.zeros((64,10)), epochs = int(100), learning_rate = lambda s : 0.01/(s+1))

m = mydataset(data=X_digits_train.to_numpy(), label=y_digits_train)
dataloader = DataLoader(m, batch_size=64, shuffle=True, num_workers=1, drop_last=True)
sgd    = SGD2(Logistic_loss(train_data, test_data), x_init= None, epochs = int(10), batch_size = 64, learning_rate = lambda s : 0.001)

gd.go(test=True)
sgd.go(dataloader, test=True)






# plot the train loss
all_history = { "GD"  : gd.history, 
                 "SGD": sgd.history}


fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="train loss")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["function_vals"],
                                name = i))
fig.show()



# plot the test loss
fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="test loss")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["test_loss"],
                                name = i))
fig.show()

100%|██████████| 10/10 [00:01<00:00,  7.88it/s]


# Problem 2: Newsgroup Dataset Optimization

Using any approach, optimize performance of logistic regression on the test set in **news.zip** and compare the performance of your approach to standard SGD. This dataset is the full-dimensional newsgroup dataset (as opposed to the compressed version you worked with previously). The $X$ matrices are stored in sparse matrix format and can be read using scipy.io.mmread. As the dataset is large and high-dimensional, you will have to decide on how best to allocate your computational resources. Try to utilize the sparsity of the data (i.e., don't just convert it to a dense matrix and spend all your time multiplying zeros). You may use any of the techniques covered in class or ideas from outside class (e.g., momentum, variance reduction, minibatches, adaptive learning rates, preprocessing). Describe your methodology and comment on what you found improved performance and why. Plot the performance (negative log likelihood) of your method against standard SGD in terms of the number of gradient evaluations. 

In [7]:
from scipy.io import mmread
import sklearn.feature_selection
import zipfile as zipfile


#sample code to load news.zip
def loadnewsdata(filename='./news.zip'):
    data={}
    with zipfile.ZipFile(filename) as z:
        for filename in z.namelist():
          if 'csv' in filename:
            data[filename] = pd.read_csv(z.open(filename), sep=' ', header=None)
          elif 'mtx' in filename:
            data[filename] = mmread(z.open(filename))
          else:
            raise Exception('unexpected filetype') 
    return data

news_dict = loadnewsdata('./news.zip')
print(news_dict.keys())
X_news_train = news_dict['X_news_train.mtx']
X_news_test = news_dict['X_news_test.mtx']
y_news_train = news_dict['y_news_train.csv'].to_numpy(dtype=int).ravel()
y_news_test = news_dict['y_news_test.csv'].to_numpy(dtype=int).ravel()

dict_keys(['X_news_test.mtx', 'X_news_train.mtx', 'y_news_test.csv', 'y_news_train.csv'])


In [8]:
#this logistic regression is for sparse csr arrays

##implemnted in uch as way to pass numpy array and you convert to crs 
##longer processing, but safe either way I think. Cleaner implemtnation
class csrLogistic_loss:
    def __init__(self, train_data, test_data, mou=0.0001):
        #all data is csr format sparse matrix
        self.X = train_data['data']
        self.Y = train_data['labels']
        self.X_test = test_data['data']
        self.Y_test = test_data['labels']
        self.N = np.shape(self.X.toarray())
        self.N_test = np.shape(self.X_test.toarray())
        self.mou = mou
        #pre_compute some quanities
        # Z_tr,Z_te=[],[]
        # for j in range(len(np.unique(self.Y))):
        #     Z_tr.append(np.sum(self.X.toarray()[np.where(self.Y==j)[0],:],axis=0))
        #     Z_te.append(np.sum(self.X_test.toarray()[np.where(self.Y_test==j)[0],:],axis=0))
        # self.Z = csr_matrix(np.asarray(Z_tr).T)
        # self.Z_test = csr_matrix(np.asarray(Z_te).T)

        #for testing we pre_compute Zs before initializing to avoid having to do this everytime I run
        self.Z =  Z
        self.Z_test = Z_test
    def eval(self, weights):
        weights = csr_matrix(weights)
        loss = (1./self.N[0])*(((weights.T @ self.Z).diagonal().sum()) +np.sum(np.log(np.sum(np.exp(-(self.X @ weights).toarray()),axis=1))))
        reg=self.mou*la.norm(weights.toarray(),'fro')**2
        return loss + reg
    def gradient(self,weights):
        return self.subgradient(weights)
    def subgradient(self,weights):
        weights = csr_matrix(weights)
        w=csr_matrix((np.exp(-(self.X @ weights).toarray()).T/np.sum(np.exp(-(self.X @ weights).toarray()),axis=1)).T)
        g_loss=(1./self.N[1])*(self.Z + (-self.X.T @ w))
        g_reg=2*self.mou*weights
        return g_loss + g_reg
    def test_eval(self, weights):
        weights = csr_matrix(weights)
        loss = (1./self.N_test[0])*(((weights.T @ self.Z_test).diagonal().sum()) +np.sum(np.log(np.sum(np.exp(-(self.X_test @ weights).toarray()),axis=1))))
        reg=self.mou*la.norm(weights.toarray(),'fro')**2
        return loss # no reg term for testing

In [46]:
#this logsitc regression for np arrays
class Logistic_loss:
    def __init__(self, train_data, test_data, mou=0.0001):
        self.X = train_data['data']
        self.Y = train_data['labels']
        self.X_test = test_data['data']
        self.Y_test = test_data['labels']
        self.N = np.shape(self.X)
        self.mou = mou
        # pre_compute some quanities
        Z_tr,Z_te=[],[]
        for j in range(len(np.unique(max(self.Y.shape[0],self.Y_test.shape[0])))):
            Z_tr.append(np.sum(self.X[np.where(self.Y==j)[0],:],axis=0))
            Z_te.append(np.sum(self.X_test[np.where(self.Y_test==j)[0],:],axis=0))
        self.Z = np.asarray(Z_tr).T
        self.Z_test = np.asarray(Z_te).T   
    def eval(self, weights):
        loss=(1./self.N[0])*(np.trace(np.dot(weights.T,self.Z))+np.sum(np.log(np.sum(np.exp(-np.dot(self.X,weights)),axis=1))))
        reg=self.mou*la.norm(weights,'fro')**2
        return loss + reg
    def gradient(self,weights):
        return self.subgradient(weights)
    def subgradient(self,weights):
        w=(np.exp(-np.dot(self.X,weights)).T/np.sum(np.exp(-np.dot(self.X,weights)),axis=1)).T
        g_loss=(1./self.N[0])*(self.Z + np.dot(-self.X.T,w))
        g_reg=2*self.mou*weights
        return g_loss + g_reg
    def test_eval(self, weights):
        loss=(1./self.N[0])*(np.trace(np.dot(weights.T,self.Z_test))+np.sum(np.log(np.sum(np.exp(-np.dot(self.X_test ,weights)),axis=1))))
        reg=self.mou*la.norm(weights,'fro')**2
        return loss # no reg term for testing


#reduce dimension

---



In [10]:
from scipy.sparse import csr_matrix


In [11]:
#this is max sum columns featurizer


# index_common_features = np.argpartition(np.array(X_news_train.sum(axis=0))[0], -500)[-500:]
# X_news_train = csr_matrix(X_news_train.toarray()[:, index_common_features])
# X_news_test  = csr_matrix(X_news_test.toarray()[:, index_common_features])

# print("test reduced features size ", X_news_test.shape)
# print("train reduced features size ", X_news_train.shape)


# import pandas as pd
# import numpy as np

# df_describe = pd.DataFrame()
# # df_describe.describe()

In [12]:
from sklearn.feature_selection import SelectKBest, chi2


ch2 = SelectKBest(chi2, k=20)
X_train = ch2.fit_transform(X_news_train, y_news_train)
X_news_train = ch2.transform(X_news_train)
X_news_test = ch2.transform(X_news_test)


In [13]:
#for testing we pre_compute Zs before initializing to avoid having to do this everytime I run
Z_tr,Z_te=[],[]
for j in range(len(np.unique(y_news_train))):
    Z_tr.append(np.sum(X_news_train.toarray()[np.where(y_news_train==j)[0],:],axis=0))
    Z_te.append(np.sum(X_news_test.toarray()[np.where(y_news_test==j)[0],:],axis=0))
Z = csr_matrix(np.asarray(Z_tr).T)
Z_test = csr_matrix(np.asarray(Z_te).T)
Z_tr,Z_te=[],[]  ##empy large list

#full GD approach


In [14]:
import scipy
from scipy.sparse.linalg import norm


# def L_M_sparse(A):
#   sm , sc = (1/A.shape[0])*norm(A,'fro')**2,(1/A.shape[0])*norm(A,1)**2
#   return sm , sc
  
train_data = {"data"  : csr_matrix(X_news_train),
              "labels": csr_matrix(y_news_train)}

test_data  = {"data"  : csr_matrix(X_news_test),
              "labels": csr_matrix(y_news_test)}

gd    = GD(csrLogistic_loss(train_data, test_data), x_init= csr_matrix(np.zeros((20,20))), epochs = int(1000), learning_rate = lambda s : 0.01/(s+1))

gd.go(test=True)

# plot the train loss
all_history = { "GD"  : gd.history }

fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="GD")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["function_vals"],
                                name = i))
fig.show()



# plot the test loss
fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="GD")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["test_loss"],
                                name = i))
fig.show()

#Stochastic gradient descend implemtned with standard GD class


In [45]:

class SGD:
    def __init__(self, function, x_init, epochs, batch_size,learning_rate):
        self.function = function
        self.x_init = x_init
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.history = {"step": [0],
                        "function_vals": [],
                        "grad_vals": [], 
                        "x_vals": [np.zeros((20,20))], 
                        "opt_dif": [],
                        "test_loss": []}      
    def go(self, dataloader, test = False):
        for _ in tqdm(range(self.epochs)):
            for i, (data, label) in enumerate(dataloader):
                train_data = {"data"  : csr_matrix(data.numpy()),
                              "labels": csr_matrix(label.numpy())}

                gd  = GD(csrLogistic_loss(train_data = train_data, test_data = test_data), x_init= None, epochs = 1, learning_rate = lambda s : 0.001)
                gd.history = self.history.copy()
                gd.go(test = test)
                self.history = gd.history.copy()




In [43]:
import scipy
from scipy.sparse.linalg import norm
from torch.utils.data import Dataset, DataLoader

class mydataset(Dataset): #accepts numpy
    def __init__(self, data, label, transform=None):
      self.data = data
      self.label = label
    def __len__(self):
      return len(self.label)
    def __getitem__(self, idx):
      data, label  = self.data[idx,:], self.label[idx]
      return data, label 
      
  
train_data = {"data"  : csr_matrix(X_news_train),
              "labels": csr_matrix(y_news_train)}
test_data  = {"data"  : csr_matrix(X_news_test),
              "labels": csr_matrix(y_news_test)}

gd    = GD(csrLogistic_loss(train_data, test_data), x_init= csr_matrix(np.zeros((20,20))), epochs = int(100), learning_rate = lambda s : 0.01/(s+1))

m = mydataset(data=X_news_train.toarray(), label=y_news_train)
dataloader = DataLoader(m, batch_size=64, shuffle=True, num_workers=1, drop_last=True)
sgd    = SGD(csrLogistic_loss(train_data, test_data), x_init= None, epochs = int(10), batch_size = 64, learning_rate = lambda s : 0.001)

gd.go(test=True)
sgd.go(dataloader, test=True)

# plot the train loss
all_history = { "GD"  : gd.history, 
                 "SGD": sgd.history}


fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="train loss")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["function_vals"],
                                name = i))
fig.show()



# plot the test loss
fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text="test loss")))
for i in all_history:
    fig.add_trace(graph.Scatter(x    = all_history[i]["step"],
                                y    = all_history[i]["test_loss"],
                                name = i))
fig.show()



  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:01<00:16,  1.83s/it]

 20%|██        | 2/10 [00:03<00:14,  1.83s/it]

 30%|███       | 3/10 [00:05<00:12,  1.82s/it]

 40%|████      | 4/10 [00:07<00:10,  1.83s/it]

 50%|█████     | 5/10 [00:09<00:09,  1.83s/it]

 60%|██████    | 6/10 [00:10<00:07,  1.82s/it]

 70%|███████   | 7/10 [00:12<00:05,  1.83s/it]

 80%|████████  | 8/10 [00:14<00:03,  1.83s/it]

 90%|█████████ | 9/10 [00:16<00:01,  1.83s/it]

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]


In [44]:
tqdm._instances.clear()